Here is the counts:

1. The number of time the Hawk took over
2. The number of total plays

In [1]:
number_of_hawk_populations = 0 + 1 + 1 + 1 + 1
number_of_plays = 1 + 1 + 1 + 1 + 2 

Thus, our simulation gives the total probability of a Hawk taking over:

In [2]:
number_of_hawk_populations / number_of_plays

0.6666666666666666

In [3]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import tqdm

def roll_n_sided_dice(n=6):
    """
    Roll a dice with n sides.
    """
    return np.random.randint(1, n + 1)

class MoranProcess:
    """
    A class for a moran process with a population of
    size N=3 using the standard Hawk-Dove Game:

    A =
       [0, 3]
       [1, 2]

    Note that this is a simulation corresponding to an
    in class activity where students roll dice.
    """
    def __init__(self, number_of_hawks=1, seed=None):

        if seed is not None:
            np.random.seed(seed)

        self.number_of_hawks = number_of_hawks
        self.number_of_doves = 3 - number_of_hawks

        self.dice_and_values_for_hawk_birth = {1: (6, {1, 2, 3}), 2: (4, {1, 2, 3})}
        self.dice_and_values_for_hawk_death = {1: (6, {1, 2}), 2: (6, {1, 2, 3, 4})}

        self.history = [(self.number_of_hawks, self.number_of_doves)]

    def step(self):
        """
        Select a hawk or a dove for birth.
        Select a hawk or a dove for death.

        Update history and states.
        """
        birth_dice, birth_values = self.dice_and_values_for_hawk_birth[self.number_of_hawks]
        death_dice, death_values = self.dice_and_values_for_hawk_death[self.number_of_hawks]

        select_hawk_for_birth = self.roll_dice_for_selection(dice=birth_dice, values=birth_values)
        select_hawk_for_death = self.roll_dice_for_selection(dice=death_dice, values=death_values)

        if select_hawk_for_birth:
            self.number_of_hawks += 1
        else:
            self.number_of_doves += 1
        if select_hawk_for_death:
            self.number_of_hawks -= 1
        else:
            self.number_of_doves -= 1

        self.history.append((self.number_of_hawks, self.number_of_doves))

    def roll_dice_for_selection(self, dice, values):
        """
        Given a dice and values return if the random roll is in the values.
        """
        return roll_n_sided_dice(n=dice) in values

    def simulate(self):
        """
        Run the entire simulation: repeatedly step through
        until the number of hawks is either 0 or 3.
        """
        while self.number_of_hawks in [1, 2]:
            self.step()
        return self.number_of_hawks
    
    def __len__(self):
        return len(self.history)

In [4]:
repetitions = 10 ** 5
end_states = []
path_lengths = []
for seed in range(repetitions):
    mp = MoranProcess(seed=seed)
    end_states.append(mp.simulate())
    path_lengths.append(len(mp))
counts = collections.Counter(end_states)
counts[3] / repetitions

0.54666

In [5]:
import nashpy as nash

A = np.array([[0, 3], [1, 2]])
game = nash.Game(A)
game

Zero sum game with payoff matrices:

Row player:
[[0 3]
 [1 2]]

Column player:
[[ 0 -3]
 [-1 -2]]

In [6]:
game.fixation_probabilities(
    initial_population=(0, 1, 1), 
    repetitions=1_000,
)

array([0.567, 0.433])